In [2]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=64286954e43b351d19b18154418659ffc28e64c72685f3bf78f8630cc350e6d2
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [3]:
from bs4 import BeautifulSoup
import requests

cont = {}
for i in range(2):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}
  url = f'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=list&&articleLimit=10&article.offset={i*10}'
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.text, 'lxml')

  contents = soup.select('tbody > tr > td.title.left > a')
  for i in contents:
    cont[i.get("title")] = 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do'+ i.get("href")
print(cont)


{'수업 중 실시간  출결 처리 시 전자출결 앱 사용 유의사항 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=462992&article.offset=10&articleLimit=10', '[10/4-10/6] 2023-2학기 군복무 중 취득학점 인정 신청 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=462248&article.offset=10&articleLimit=10', '2023학년도 2학기 취업진로상담 교과목 시행안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=461550&article.offset=10&articleLimit=10', '2023-2학기 공결신청 방법 및 유의사항 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=456000&article.offset=10&articleLimit=10', '[재공지][학생성공처]2023학년도 KIT역량장학금(K-WEGO) 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=449405&article.offset=10&articleLimit=10', '[LINC3.0] 2023년 국내 현장견학 지원금 신청 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=427508&article.offset=10&articleLimit=10', '★ 학적부 기재사항 현행화 요청 및 정정 신청 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=425744&article.offset=1

In [4]:
import re

sen_list = []
i = 0
count = len(cont)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}
for link in cont.values():
  print(f"크롤링 중({count-i}개)",end = " ")
  i += 1
  r = requests.get(link, headers=headers)
  s = BeautifulSoup(r.text, 'lxml')

  c = s.select("#jwxe_main_content > div.contents-wrapper > div.board-area.ko.board.view > div.board-wrap > div > div.board-contents > p > span")
  le = ""
  for word in c:
    le += word.text.strip()
  sen_list.append(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s().,-/:~""'']", " ", le))

크롤링 중(28개) 크롤링 중(27개) 크롤링 중(26개) 크롤링 중(25개) 크롤링 중(24개) 크롤링 중(23개) 크롤링 중(22개) 크롤링 중(21개) 크롤링 중(20개) 크롤링 중(19개) 크롤링 중(18개) 크롤링 중(17개) 크롤링 중(16개) 크롤링 중(15개) 크롤링 중(14개) 크롤링 중(13개) 크롤링 중(12개) 크롤링 중(11개) 크롤링 중(10개) 크롤링 중(9개) 크롤링 중(8개) 크롤링 중(7개) 크롤링 중(6개) 크롤링 중(5개) 크롤링 중(4개) 크롤링 중(3개) 크롤링 중(2개) 크롤링 중(1개) 

In [5]:
sf_list = []
for i in sen_list:
  a = list(i.split("."))
  for i in range(len(a)):
    a[i] = a[i] + "\n"
  sf_list.append(a)


In [6]:
crawling = dict(zip(cont.keys(),sf_list))

In [7]:
"""del_k = []
for title, text in crawling.items():
  if text == ['']:
    del_k.append(title)
for i in del_k:
  del crawling[i]"""

"del_k = []\nfor title, text in crawling.items():\n  if text == ['']:\n    del_k.append(title)\nfor i in del_k:\n  del crawling[i]"

In [8]:
#구글드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
for title, text in crawling.items():
  title = title.replace("/","-")
  if text == ['']:
    continue
  path = f'/content/drive/My Drive/Colab Notebooks/data/kit_text/{title}.txt'
  with open(path,'w') as f:
    for line in text:
      f.write(line)

In [10]:
!pip install langchain unstructured openai chromadb Cython tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.4 MB/s eta 0:00:0

In [11]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c197e396f2cc6b6a7d601f7f73d13e1b15e2dd07fb711479a6c692c3c7783ff9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [12]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

from langchain.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings

In [13]:
embeddings = HuggingFaceEmbeddings()

In [14]:
fn_dir = "/content/drive/My Drive/Colab Notebooks/data/kit_text"

In [15]:
loader = DirectoryLoader(fn_dir)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

len(documents), len(docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


(116, 108)

In [16]:
db = Chroma.from_documents(docs, embedding=embeddings,
                           persist_directory="kit_index_hf")
db.persist()

In [23]:
query = "2학기"
doc = db.similarity_search(query)

In [24]:
doc[:5]

[Document(page_content='재원 후가 배정시 재안내드리겠습니다', metadata={'source': '/content/drive/My Drive/Colab Notebooks/data/kit_text/[2023년도 국립대학 육성사업] 대학원생 학술활동 참가비 지원 마감 안내(재원 소진).txt'}),
 Document(page_content='2023학년도2학기 등록금 납부 안내(재학생,복학예정자,휴학생) 등록 등록금 고지서 출력 바로가기 고지서 발급일자:2023\n\n8\n\n17\n\n(목)부터 예정(수업연한초과자는2023\n\n9\n\n14\n\n(수)부터 예정) 고지서 발급절차: kit포탈-원스톱서비스-학사-등록금 고지내역조회/출력 고지서 발급대상:재학생,복학예정자,휴학생 등록금 납부기간:2023\n\n8\n\n21\n\n(월)~ 8\n\n24\n\n(목)  매일09:00~16:00  수업연한초과자는2023\n\n9\n\n14\n\n(목)~ 9\n\n15\n\n(금)  매일09:00~16:00,농협은행  등록금 납부은행:농협은행,대구은행,기업은행1\n\n등록금 납부방법1\n\n등록금 납부방법 은행에 직접 방문하여 납부할 경우-고지서 지참(은행 영업시간:9시~16시)-등록금 납부금액이 0원 (전액면제자)인 경우 납부 방법 은행에 직접 방문하여 등록절차(등록처리)이행 농협은행인터넷뱅킹-공과금-대학등록금에서0원 이체 개인별 가상계좌번호로 등록금 납부/입금(인터넷뱅킹,폰뱅킹 등 이용시)-받는 사람 이름을(학생본인이름)필히 확인하세요\n\n보내는 사람은 학생이름이 아니라도 상관없습니다\n\n(보호자 통장에서 이체 납부 가능합니다\n\n)-가상계좌번호는 매학기 마다 변경될 수 있습니다\n\n2\n\n수업연한초과(학기초과)등록자인 경우4\n\n수업연한초과(학기초과)등록자인 경우 수강신청 정정기간 이후 등록기간 설정 등록금 납부기간: 2023\n\n9\n\n14